In [1]:
import sys
import os
import pandas as pd 
from datetime import date, timedelta
# Adiciona a pasta raiz do projeto (onde está a pasta src) ao sys.path
sys.path.append(os.path.abspath(os.path.join("..")))
from src.etl.webscrapping import gerar_tabela_cotacao_diaria_ibovespa
from src.etl.s3 import carregar_dados_parquet_particao_diaria_s3
from src.config.logging_config import setup_logging

In [2]:
# configuração do logging
setup_logging()


In [23]:
tbl_carteira_ibovespa = pd.read_csv("../data/external/tbl_acoes_ibovespa_corrigido.csv", sep=';')
tbl_carteira_ibovespa['ticker'] = tbl_carteira_ibovespa['codigo'] + '.SA'
tbl_carteira_ibovespa.to_csv("../data/external/tbl_acoes_ibovespa.csv", sep=';', index=False)
tbl_carteira_ibovespa.head()

,codigo,tipo,empresa,slug,setor,industria,ticker
0,ABEV3,ON,Ambev S/A,ambev-abev3,Consumo não-cíclico,Bebidas,ABEV3.SA
1,ALOS3,ON,Allos,alos3,Imóveis,Atividades Imobiliárias,ALOS3.SA
2,ASAI3,ON,Assaí­ Atacadista,assai-asai3,Consumo não-cíclico,Varejo de alimentos e medicamentos,ASAI3.SA
3,AURE3,ON,Auren,auren-energia-aure3,Utilidades,Empresas independentes e concessionárias elétr...,AURE3.SA
4,AZZA3,ON,Azzas 2154,azzas-2154-azza3,Consumo cíclico,Varejistas especializados,AZZA3.SA


In [3]:
hoje = date.today() - timedelta(days=4)
hoje

datetime.date(2025, 7, 28)

In [4]:
# teste de execução da função de raspagem dos dados
tbl_carteira_ibovespa = pd.read_csv("../data/external/tbl_acoes_ibovespa.csv", sep=';')
tbl_cotacao_diaria = gerar_tabela_cotacao_diaria_ibovespa(lista_acoes = tbl_carteira_ibovespa['ticker'], 
                                     data_cotacao= hoje ,periodo='1d')
tbl_cotacao_diaria

2025-08-01 16:38:35,958 | INFO | src.etl.webscrapping | Iniciando o processo de captação dos dados.
2025-08-01 16:38:38,468 | INFO | src.etl.webscrapping | Dados coletados para: ABEV3.SA
2025-08-01 16:38:40,381 | INFO | src.etl.webscrapping | Dados coletados para: ALOS3.SA
2025-08-01 16:38:42,231 | INFO | src.etl.webscrapping | Dados coletados para: ASAI3.SA
2025-08-01 16:38:44,107 | INFO | src.etl.webscrapping | Dados coletados para: AURE3.SA
2025-08-01 16:38:46,046 | INFO | src.etl.webscrapping | Dados coletados para: AZZA3.SA
2025-08-01 16:38:48,305 | INFO | src.etl.webscrapping | Dados coletados para: B3SA3.SA
2025-08-01 16:38:50,221 | INFO | src.etl.webscrapping | Dados coletados para: BBAS3.SA
2025-08-01 16:38:52,220 | INFO | src.etl.webscrapping | Dados coletados para: BBDC3.SA
2025-08-01 16:38:54,166 | INFO | src.etl.webscrapping | Dados coletados para: BBDC4.SA
2025-08-01 16:38:56,112 | INFO | src.etl.webscrapping | Dados coletados para: BBSE3.SA
2025-08-01 16:38:58,335 | INFO

c:\Users\ederr\Documents\PROJETOS_PYTHON\projetos_fiap\Fase_2\etl-aws\src\etl\webscrapping.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tbl_cotacao_diaria = pd.concat(lista_df)


,ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,trailingPE,priceToBook,returnOnEquity,profitMargins,revenueGrowth,dividendYield,totalDebt,freeCashflow
data_cotacao,,,,,,,,,,,,,,,,
2025-07-28,ABEV3,12.550000,12.680000,12.220000,12.310000,26548400,0.0,0.0,13.236560,2.037742,0.16240,0.15742,0.110,7.41,3218170112,18302226432
2025-07-28,ALOS3,21.660000,21.830000,21.410000,21.559999,4399300,0.0,0.0,13.391304,0.803728,0.06593,0.30534,0.007,5.63,6455017984,855961344
2025-07-28,ASAI3,9.540000,9.840000,9.480000,9.670000,11300500,0.0,0.0,15.852459,2.421738,0.16384,0.01099,0.077,1.14,25693999104,-144307008
2025-07-28,AURE3,9.360000,9.850000,9.360000,9.630000,5052100,0.0,0.0,160.666670,0.760192,0.00539,0.00060,1.113,0.62,26161764352,504947872
2025-07-28,AZZA3,36.380001,36.849998,34.810001,35.080002,1508500,0.0,0.0,14.088354,0.928067,0.07151,0.03863,1.515,6.16,4073274880,-1987710976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-28,CPLE6,12.000000,12.080000,11.900000,11.990000,10555400,0.0,0.0,15.177215,1.355109,0.09577,0.12729,0.088,5.12,19138037760,-4095745536
2025-07-28,TIMS3,20.889999,20.990000,20.520000,20.740000,3849000,0.0,0.0,13.919463,2.052246,0.13872,0.13331,0.049,6.31,15795061760,6524463616
2025-07-28,VAMO3,3.920000,4.040000,3.890000,3.910000,12760200,0.0,0.0,6.627119,1.678832,0.17202,0.06173,0.236,15.30,16508325888,2585804288


In [5]:
carregar_dados_parquet_particao_diaria_s3(df=tbl_cotacao_diaria,data_cotacao=hoje)

2025-08-01 16:41:35,306 | INFO | src.etl.s3 | Início da transformação e carregamento dos dados no formato parquet para S3.
2025-08-01 16:41:35,306 | INFO | src.etl.s3 | Variável de Ambiente CAMINHO_CSV (URL S3 completo): s3://bucket-ibovespa-302907108883/referencias/tbl_acoes_ibovespa.csv
2025-08-01 16:41:35,307 | INFO | src.etl.s3 | Bucket extraído para S3: bucket-ibovespa-302907108883
2025-08-01 16:41:35,308 | INFO | src.etl.s3 | Caminho S3 de destino: s3://bucket-ibovespa-302907108883/raw/ano=2025/mes=07/dia=28/cotacao_ibovespa.parquet
2025-08-01 16:41:36,674 | INFO | src.etl.s3 | Arquivo salvo com sucesso em s3://bucket-ibovespa-302907108883/raw/ano=2025/mes=07/dia=28/cotacao_ibovespa.parquet


True